In [252]:
using ITensors
using ITensorGLMakie

First we are going to define the two qubits. We want general product states $\ket{\psi_a}\otimes \ket{\psi_b}$

In [253]:
αa = 1; βa = 1; # First qubit parameters
αb = 0; βb = 1; # Second qubit parameters

ia = Index(2, "Qubit,i1");
ib = Index(2, "Qubit,i2");


ket0a = state("0",ia);
ket0b = state("0",ib);
ket1a = state("1",ia);
ket1b = state("1",ib);

# Define qubit states
ψa = normalize(αa*ket0a + βa*ket1a);
ψb = normalize(αb*ket0b + βb*ket1b);

# Compute the product state
ψ = ψa * ψb;

Compute density matrix and transform to MPS

In [254]:
# Density matrix:
ρ = ψ * dag(ψ')

#Transform to MPS (Probably it's better to start with MPS instead of converting)
sites = (ia, ib)

ρMPO = toMPO(ρ, sites)

MPO
[1] ((dim=2|id=850|"Qubit,i1"), (dim=2|id=138|"Link,n=1"))
[2] ((dim=2|id=138|"Link,n=1"), (dim=2|id=384|"Qubit,i2"), (dim=2|id=850|"Qubit,i1")', (dim=2|id=384|"Qubit,i2")')


In [255]:
@visualize ρMPO # Why I cannot visualize?

nothing

MPO
[1] ((dim=2|id=850|"Qubit,i1"), (dim=2|id=138|"Link,n=1"))
[2] ((dim=2|id=138|"Link,n=1"), (dim=2|id=384|"Qubit,i2"), (dim=2|id=850|"Qubit,i1")', (dim=2|id=384|"Qubit,i2")')


Now I will define the quantum channels. We can define the Kraus ops with indices ia,ib,ia',ib'

In [256]:
# First define the operators we need
Ida = op("Id",ia);
Idb = op("Id",ib);
Xa = op("X",ia);
Xb = op("X",ib);
Ya = op("Y",ia);
Yb = op("Y",ib);
Za = op("Z",ia);
Zb = op("Z",ib);

projPPa = (Ida + Xa)/ 2; # projPP and projMM are the projectors on the plus and minus states (X eigenstates)
projPPb = (Idb + Xb)/ 2;
projMMa = (Ida - Xa)/ 2;
projMMb = (Idb - Xb)/ 2;

Sa = op("S",ia);
Sb = op("S",ib);

In [257]:
# Now we define the MPOs (not sure if this is needed)
sites = [ia,ib]

II = MPO(sites)
II[1] = Ida
II[2] = Idb

XX = MPO(sites)
XX[1] = Xa
XX[2] = Xb

ZZ = MPO(sites)
ZZ[1] = Za
ZZ[2] = Zb

ch = Array{MPO}(undef,8)
for i in 1:8
    ch[i] = MPO(sites)
end

ch[1][1] = projPPa    
ch[1][2] = dag(Sb)

ch[2][1] = dag(Sa)
ch[2][2] = projPPb    

ch[3][1] = projPPa    
ch[3][2] = Sb       

ch[4][1] = Sa
ch[4][2] = projPPb    

ch[5][1] = projMMa    
ch[5][2] = dag(Sb)

ch[6][1] = dag(Sa)
ch[6][2] = projMMb    

ch[7][1] = projMMa    
ch[7][2] = Sb       

ch[8][1] = Sa
ch[8][2] = projMMb;
XX

MPO
[1] ((dim=2|id=850|"Qubit,i1")', (dim=2|id=850|"Qubit,i1"))
[2] ((dim=2|id=384|"Qubit,i2")', (dim=2|id=384|"Qubit,i2"))


In [258]:
ch[1] = replaceprime(ch[1], (0 => 2))
ch[2] = replaceprime(ch[2], (0 => 2))
ch[3] = replaceprime(ch[3], (0 => 2))
ch[4] = replaceprime(ch[4], (0 => 2))
ch[5] = replaceprime(ch[5], (0 => 2))
ch[6] = replaceprime(ch[6], (0 => 2))
ch[7] = replaceprime(ch[7], (0 => 2))
ch[8] = replaceprime(ch[8], (0 => 2))
signsch = [1,1,-1,-1,-1,-1,1,1]

II = replaceprime(II, (0 => 2))
XX = replaceprime(XX, (0 => 2));



In [259]:
XX

MPO
[1] ((dim=2|id=850|"Qubit,i1")', (dim=2|id=850|"Qubit,i1")'')
[2] ((dim=2|id=384|"Qubit,i2")', (dim=2|id=384|"Qubit,i2")'')


In [260]:
XX*ρMPO*XX

MPO
[1] ((dim=2|id=850|"Qubit,i1"), (dim=2|id=850|"Qubit,i1")', (dim=1|id=248|"Link,l=1"))
[2] ((dim=2|id=384|"Qubit,i2"), (dim=2|id=384|"Qubit,i2")', (dim=1|id=248|"Link,l=1"))


In [261]:
function ε(ρ, θ)
    ερ = cos(θ/2)^2 * contract(XX, contract(ρ, XX))
    ερ = ερ + sin(θ/2)^2 * contract(II, contract(ρ, II))
    for i in 1:8
        ερ = ερ + (1/2) * signsch[i] * sin(θ) * contract(ch[i], contract(ρ, ch[i]))
    end
    return ερ
end


ε (generic function with 2 methods)

In [262]:
θ = pi/3
ερMPO = ε(ρMPO,θ)

MPO
[1] ((dim=2|id=850|"Qubit,i1")', (dim=2|id=850|"Qubit,i1"), (dim=2|id=975|"Link,l=1"))
[2] ((dim=2|id=384|"Qubit,i2")', (dim=2|id=384|"Qubit,i2"), (dim=2|id=975|"Link,l=1"))


In [263]:
oprtrs = ερMPO*ZZ'

MPO
[1] ((dim=2|id=850|"Qubit,i1"), (dim=2|id=850|"Qubit,i1")'', (dim=1|id=449|"Link,l=1"))
[2] ((dim=2|id=384|"Qubit,i2"), (dim=2|id=384|"Qubit,i2")'', (dim=1|id=449|"Link,l=1"))


In [264]:
ket00 = MPS(ket0a * ket0b,sites)
ket11 = MPS(ket1a * ket1b,sites)
ket01 = MPS(ket0a * ket1b,sites)
ket10 = MPS(ket1a * ket0b,sites)

MPS
[1] ((dim=2|id=850|"Qubit,i1"), (dim=2|id=856|"Link,n=1"))
[2] ((dim=2|id=856|"Link,n=1"), (dim=2|id=384|"Qubit,i2"))


In [265]:
expval = inner(ket00, oprtrs,ket00) + inner(ket01, oprtrs,ket01) + inner(ket10, oprtrs,ket10) + inner(ket11, oprtrs,ket11) 

-1.1102230246251565e-16 + 0.0im